In [2]:
from src.env_loader import load_api_key
load_api_key()

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI


# If there is no environment variable set for the API key, you can pass the API
# key to the parameter `google_api_key` of the `ChatGoogleGenerativeAI` function:
# `google_api_key="key"`.
llm_gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85)

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



In [19]:
import requests
from langchain.document_loaders import TextLoader, WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
import bs4

# url = "https://raw.githubusercontent.com/hwchase17/chroma-langchain/master/state_of_the_union.txt"
# res = requests.get(url)
# with open("state_of_the_union.txt", "w") as f:
#     f.write(res.text)

# Load the data
# loader = TextLoader("./state_of_the_union.txt")
# documents = loader.load()

# Chunk the data
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=10)
# chunks = text_splitter.split_documents(documents)

# FETCH DATA FROM WEB
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
chunks = loader.load_and_split(text_splitter)

Created a chunk of size 2159, which is longer than the specified 200
Created a chunk of size 250, which is longer than the specified 200
Created a chunk of size 975, which is longer than the specified 200
Created a chunk of size 779, which is longer than the specified 200
Created a chunk of size 532, which is longer than the specified 200
Created a chunk of size 225, which is longer than the specified 200
Created a chunk of size 557, which is longer than the specified 200
Created a chunk of size 1317, which is longer than the specified 200
Created a chunk of size 549, which is longer than the specified 200
Created a chunk of size 1112, which is longer than the specified 200
Created a chunk of size 1043, which is longer than the specified 200
Created a chunk of size 352, which is longer than the specified 200
Created a chunk of size 228, which is longer than the specified 200
Created a chunk of size 279, which is longer than the specified 200
Created a chunk of size 575, which is longer

In [20]:
len(chunks)

66

In [23]:
chunks

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.', metadata={'source': './state_of_the_union.txt'}),
 Document(page_content='Last year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.', metadata={'source': './state_of_the_union.txt'}),
 Document(page_content='With a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.', metadata={'source': './state_of_the_union.txt'}),
 Document(page_content='Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.', metadata={'source': './state_of_the_union.txt'}),
 Document(page_content='He thought he

In [24]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import LanceDB
import lancedb

openai_embed = OpenAIEmbeddings()

# Setup lancedb
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "raga_eval",
    data=[{"vector": openai_embed.embed_query("Hello World"), "text": "Hello World"}],
    mode="overwrite",
)

# Populate vector database
vectorstore = LanceDB.from_documents(
    client=table, documents=chunks, embedding=openai_embed, by_text=False
)

# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()

In [25]:
# from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Define LLM
# llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Define Prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm_gemini
    | StrOutputParser()
)

In [26]:
rag_chain.invoke("What is the state of the union?")

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


KeyboardInterrupt: 

In [10]:
from datasets import Dataset
import time

questions = [
    "What did the president say about Justice Breyer?",
    "What did the president say about Intel's CEO?",
    "What did the president say about gun violence?",
]
ground_truth = [
    "The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service.",
    "The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion.",
    "The president asked Congress to pass proven measures to reduce gun violence.",
]
answers = []
contexts = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    time.sleep(2)
    contexts.append(
        [docs.page_content for docs in retriever.get_relevant_documents(query)]
    )

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth,
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


KeyboardInterrupt: 

In [10]:
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    faithfulness,
    answer_similarity,
    context_precision,
    context_utilization,
    context_recall,
    context_relevancy,
    answer_relevancy,
    context_entity_recall,
)


# evaluating dataest on listed metrics
result = evaluate(
    dataset=dataset,
    metrics=[
        answer_correctness,
        faithfulness,
        answer_similarity,
        context_precision,
        context_utilization,
        context_recall,
        context_relevancy,
        answer_relevancy,
        context_entity_recall,
    ],
)


df = result.to_pandas()

Evaluating:   0%|          | 0/27 [00:00<?, ?it/s]

In [11]:
# evaluation metrics
df

,question,answer,contexts,ground_truth,answer_correctness,faithfulness,answer_similarity,context_precision,context_utilization,context_recall,context_relevancy,answer_relevancy,context_entity_recall
0,What did the president say about Justice Breyer?,The president honored Justice Stephen Breyer a...,"[And I did that 4 days ago, when I nominated C...",The president said that Justice Breyer has ded...,0.394747,0.857143,0.912321,1.0,1.0,1.0,0.100000,0.841583,0.333333
1,What did the president say about Intel's CEO?,"The president said that Intel’s CEO, Pat Gelsi...","[Intel’s CEO, Pat Gelsinger, who is here tonig...",The president said that Pat Gelsinger is ready...,0.620013,0.000000,0.980103,1.0,1.0,1.0,0.090909,0.915200,0.750000
2,What did the president say about gun violence?,The president asked Congress to pass proven me...,[And I ask Congress to pass proven measures to...,The president asked Congress to pass proven me...,0.609959,1.000000,0.939837,1.0,1.0,1.0,0.250000,0.905776,0.666667
